# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [17]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release)]
        trad_releases = [release for release in project.main_releases if is_trad_release(release)]

        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        for srelease in project.main_releases:
            # release = srelease.release
            cycle_days = delta2days(srelease.cycle)
            delay = delta2days(srelease.delay)
            prev_release = srelease.prev_main_release

            if srelease.commits:
                last_commit_time = srelease.commits.last(lambda c: c.committer_time).committer_time
                first_commit_time = srelease.commits.first(lambda c: c.author_time).author_time
                duration = delta2days(last_commit_time - first_commit_time)
                # duration = delta2days(srelease.time - srelease.commits.first(lambda c: c.author_time).author_time)
            else:
                duration = 0
                
            if srelease.commits and prev_release:
                if prev_release.commits:
                    prev_release_time = prev_release.commits.last(lambda c: c.committer_time).committer_time
                else:
                    prev_release_time = prev_release.time
                    
                commits_in_advance = [commit for commit in srelease.commits if commit.author_time < prev_release_time]
            else: 
                commits_in_advance = []
                
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle': srelease.cycle,
                'cycle_days': cycle_days,
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': delay,
                'duration': duration,
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [19]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [20]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1)

In [21]:
projects = pd.DataFrame(project_results)
projects.sample(1)

,project,prefixes,prefixes_names,main_releases,rapid_releases,trad_releases,patches,releases,time
0,robinvdvleuten/vuex-persistedstate,1,v,21,9,7,14,35,0 days 00:00:00.041606


In [22]:
releases = pd.DataFrame(release_results)
releases.sample(10)

,project,release,previous_release,cycle,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
2,robinvdvleuten/vuex-persistedstate,v0.3.0,v0.2.0,0 days 05:44:56,0.239537,True,False,9,0,0.001794,0.237488,0
15,robinvdvleuten/vuex-persistedstate,v2.7.0,v2.6.0,2 days 20:11:41,2.841447,True,False,11,0,2.796389,0.045058,1
7,robinvdvleuten/vuex-persistedstate,v1.4.0,v1.3.0,40 days 22:26:50,40.935301,True,False,4,0,6.482280,34.452755,1
1,robinvdvleuten/vuex-persistedstate,v0.2.0,v0.1.0,0 days 17:24:28,0.725324,True,False,4,0,-0.003275,0.728368,0
12,robinvdvleuten/vuex-persistedstate,v2.4.0,v2.3.0,17 days 14:43:39,17.613646,True,False,7,0,15.544097,2.069502,2
16,robinvdvleuten/vuex-persistedstate,v3.0.0,v2.7.0,146 days 20:46:34,146.865671,False,True,14,0,97.916586,48.949086,1
5,robinvdvleuten/vuex-persistedstate,v1.2.0,v1.1.0,80 days 23:23:11,80.974433,False,False,5,0,4.941725,76.032384,0
3,robinvdvleuten/vuex-persistedstate,v1.0.0,v0.3.0,56 days 19:05:59,56.795822,False,False,5,0,56.773715,0.021979,0
13,robinvdvleuten/vuex-persistedstate,v2.5.0,v2.4.0,111 days 09:24:16,111.391852,False,True,8,0,3.153750,108.238102,4
20,robinvdvleuten/vuex-persistedstate,v4.1.0,v4.0.0,64 days 01:27:23,64.060683,False,False,5,0,0.008368,64.052315,0


In [9]:
len(projects)

1665

In [10]:
len(releases)

52836

In [11]:
len(releases.query("rapid_release == True"))

29306

In [12]:
len(releases.query("trad_release == True"))

11296

In [13]:
len(releases.query("rapid_release == False and trad_release == False"))

12234

In [14]:
releases['commits'].sum()

5520217

In [15]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [16]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)